In [2]:
import numpy as np
import math as m

### Scratch work

In [65]:
# Problem 4.20 blaze it
P = 5e6
T = 50e3
ou = 250e6
oup = 520e6
do = 0.21
di = 0.2

c = do/2
r = (do)/2
t = (do-di)/2
J = 2*np.pi*(do**3)*t

tau_xy = T*c/J
ox = P*r/(2*t)
oy = P*r/(t)

print(ox, oy, tau_xy)

print(1/(284.9285/420 - -64.9285/900))

52500000.0000001 105000000.0000002 18044778.128332842
1.332366979547321


In [68]:
20*.125*12/(8*.01)/2

187.5

### 3D Principal Stresses (x, y, z coordinate system only)
Fill out matrix below as provided in the question. Relevant parameters will be printed.

In [28]:
stress_mat = np.array([[ 200, 150, 0],
                       [ 150, 20, 0],
                       [ 0, 0, 0]])

principal_stresses(stress_mat[0, 0], stress_mat[0, 1], stress_mat[0, 2], stress_mat[1, 1], stress_mat[1, 2], stress_mat[2, 2])

Invariants:  220 -18500 0.0
Principal stresses:  [284.928556845359, 0.0, -64.92855684535901]
Direction:  [0.8701999067534787, 0.4926988149835931, 0.0]
Octahedral normal stress: 73.33 MPa
Octahedral shear stress: 151.95 MPa
Abs max shearing stress: 174.93 MPa


In [6]:
def principal_stresses(sigma_x, tau_xy, tau_xz, sigma_y, tau_yz, sigma_z):
    # Calculate principal stresses
    stress_matrix = np.array([[sigma_x, tau_xy, tau_xz],
                             [tau_xy, sigma_y, tau_yz],
                             [tau_xz, tau_yz, sigma_z]])
    i1 = sigma_x + sigma_y + sigma_z
    i2 = sigma_x*sigma_y + sigma_y*sigma_z + sigma_x*sigma_z - tau_xy**2 - tau_yz**2 - tau_xz**2
    i3 = np.linalg.det(stress_matrix)
    sigma_p = sorted(np.roots([1, -i1, i2, -i3]), reverse=True)

    # Calculate directional cosines (l, m, n)
    a = np.linalg.det(
        np.array([[sigma_y-sigma_p[0], tau_yz],
                  [tau_yz, sigma_z-sigma_p[0]]]))
    b = -np.linalg.det(
        np.array([[tau_xy, tau_yz],
                  [tau_xz, sigma_z-sigma_p[0]]]))
    c = np.linalg.det(
        np.array([[tau_xy, sigma_y-sigma_p[0]],
                  [tau_xz, tau_yz]]))
    k = 1/m.sqrt(a**2 + b**2 + c**2)
    l1 = a*k
    m1 = b*k
    n1 = c*k
    dir_cos = [l1, m1, n1]

    sigma_oct = sum(sigma_p)/3
    tau_oct = m.sqrt((sigma_p[0]-sigma_p[1])**2 + (sigma_p[1]-sigma_p[2])**2 + (sigma_p[2]-sigma_p[0])**2)/3
    tau_max = (sigma_p[0]-sigma_p[2])/2

    # print('Prime CSYS stresses: ', prime_stress_matrix)
    print('Invariants: ', i1, i2, i3)
    print('Principal stresses: ', sigma_p)
    print('Direction: ', dir_cos)
    print('Octahedral normal stress: %.2f MPa' % sigma_oct)
    print('Octahedral shear stress: %.2f MPa' % tau_oct)
    print('Abs max shearing stress: %.2f MPa' % tau_max)